In [115]:
########################### Libraries ####################################
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
import astropy
import pandas as pd
import numpy as np 
import matplotlib.pyplot as plt
import json
import scipy
from scipy import stats
from scipy.signal import argrelextrema
import matplotlib.backends.backend_pdf
from gatspy import datasets, periodic
from scipy.signal import find_peaks
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from xgboost import XGBClassifier
from sklearn.metrics import average_precision_score
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import roc_curve, auc,recall_score,precision_score
from sklearn.metrics import accuracy_score
import timeit
from astropy.timeseries import LombScargle
from sklearn.metrics import roc_auc_score
from sklearn.metrics import log_loss  
from sklearn.metrics import f1_score
from sklearn.metrics import precision_score
import requests
import urllib
import os
from os import path
from numpy import mean
from sklearn.datasets import make_classification
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.tree import DecisionTreeClassifier
from imblearn.pipeline import Pipeline
from imblearn.over_sampling import SMOTE
from imblearn.under_sampling import RandomUnderSampler

In [116]:
data_dir = os.getcwd()
directory = "catalogs"
subdirectoryres='res'
path = os.path.join(data_dir, directory)
pathres= os.path.join(path, subdirectoryres) 

In [117]:
pathres

'C:\\Users\\rtut6\\Desktop\\catalogs\\res'

### df1 - homogenuous set in zg , df2 - homogenuous set in zr, df3 - overall (zg(min)+zr(min))

In [118]:
# df1=pd.read_csv('df1_homogen_zg.csv')
# df2=pd.read_csv('df1_homogen_zr.csv')
# df3=pd.read_csv('df1_homogen_summ.csv')

def openandfilter(name):
    df=pd.read_csv(name, sep='\t')
    df.drop('Unnamed: 0',axis=1,inplace=True)
    df.dropna(inplace=True)
    df['Name']=df.apply(lambda x: int(x['Name']),axis=1)
    df=df.reset_index()
    return df

df1=openandfilter('homogen_zg_example1.csv')
df2=openandfilter('homogen_zr_example1.csv')
df3=openandfilter('homogen_summ_example1.csv')

In [96]:
# df1=pd.read_csv('homogen_summ_example.csv', sep='\t')
# df2=pd.read_csv('df1_homogen_zr.csv')
# df3=pd.read_csv('df1_homogen_summ.csv')
df3.isnull().sum().sum()

0

In [5]:
# df1=df1[df1['Nobs']>30]
# df1=df1.iloc[:10000]
# df1=df1.reset_index()

# df2=df2[df2['Nobs']>30]
# df2=df2.iloc[:10000]
# df2=df2.reset_index()


# df3=df3[df3['Nobs']>30]
# df3=df3.iloc[:10000]
# df3=df3.reset_index()

In [6]:
# df2[df2['target']==1].shape[0]

### Features - features generation, df- homogenuous df with 'Name' columns (names of files in the folder)

### Metrics - function to receive final results

In [204]:
def Features(var_type,df):
    start = timeit.default_timer()
    df['period-0']=0.
    df['power-0']=0.
    df['period-1']=0.
    df['power-1']=0.
    df['period-2']=0.
    df['power-2']=0.
    df['period-3']=0.
    df['power-3']=0.
    df['period-4']=0.
    df['power-4']=0.
    df['abs_deviation_mag']=0.
    df['abs_deviation_pow']=0.
    df['freq_ratio_10']=0.
    df['freq_ratio_20']=0.    
    df['mean_magn']=0.
    df['amplitude']=0.
    df['trend_angle']=0.
    df['skew']=0.
    df['std']=0.
    df['target']=0
#     df=df.iloc[:n]
    names=df['Name'].tolist()
    for i in range(df.shape[0]):
        if df['Type'][i]==var_type:
            df['target'][i]=1
        else:
            df['target'][i]=0
    # model = periodic.LombScargleAstroML(fit_period=False)    
    # model.optimizer.set(period_range=(1, 70), first_pass_coverage=30)    
    k=0
    for item in names:
        try:
            pathname=os.path.join(pathres,str(item)+".txt")
            with open(pathname) as f:
                res = f.read()#.replace("'", '"')
                res=json.loads(res)
                obj_mjd=[]
                obj_mag=[]
                obj_magerr=[]

                for obj in res.keys():
                    for ii in res[obj]['lc']:
                        obj_mjd.append(ii['mjd'])
                        obj_mag.append(ii['mag'])
                        obj_magerr.append(ii['magerr'])

                    frequency, power = LombScargle(obj_mjd, obj_mag, obj_magerr).autopower()
                    peaks=list(find_peaks(power))[0].tolist()
                    df['period-0'][k]=1/frequency[peaks[0]]
                    df['power-0'][k]=power[peaks[0]]
                    df['period-1'][k]=1/frequency[peaks[1]]
                    df['power-1'][k]=power[peaks[1]]
                    df['period-2'][k]=1/frequency[peaks[2]]
                    df['power-2'][k]=power[peaks[2]]
                    df['period-3'][k]=1/frequency[peaks[3]]
                    df['power-3'][k]=power[peaks[3]]
                    df['period-4'][k]=1/frequency[peaks[4]]
                    df['power-4'][k]=power[peaks[4]]
                    df['abs_deviation_mag'][k]=stats.median_absolute_deviation(obj_mag)
                    df['abs_deviation_pow'][k]=stats.median_absolute_deviation(power)
                    df['freq_ratio_10'][k]=frequency[peaks[1]]/frequency[peaks[0]]
                    df['freq_ratio_20'][k]=frequency[peaks[2]]/frequency[peaks[0]]
                    df['mean_magn'][k] = np.array(obj_mag).mean()
                    df['amplitude'][k] = (np.array(obj_mag).max() - np.array(obj_mag).min())/2
                    df['trend_angle'][k] = np.polyfit(np.array(obj_mjd), np.array(obj_mag), 1)[0] 
                    df['skew'][k] = scipy.stats.skew(np.array(obj_mag))
                    df['std'][k] = np.array(obj_mag).std()
                    k=k+1
                           
        except:
            print('error')
    return df                
    #           indices = find_peaks(power)
    #           model.fit(obj_mjd, obj_mag, obj_magerr)
    #           for i in range(3):
    #               df['period-'+str(i)][k]=np.round(model.find_best_periods(n_periods=5,return_scores=True)[0][i],3)
    #                   df['power-'+str(i)][k]=power[int(indices[0][i])]

#         except:
#             print(1)


def Metrics(df):
    n=sum(df['target'].tolist())
    dff=df[df['target']==0].iloc[:n]
    df=df[df['target']==1]
    r=[df,dff]
    df=pd.concat(r)
    print(df.shape)
#     X = np.array(df1[['period-0','period-1','period-2','period-3','period-4','abs_deviation_mag','abs_deviation_pow','freq_ratio_10', 'freq_ratio_20','mean_magn','amplitude','trand_angle','skew','std']])
    X = np.array(df[['period-0','power-0','period-1','power-1','period-2','power-2','period-3','power-3','period-4','power-4','abs_deviation_mag','abs_deviation_pow','freq_ratio_10','freq_ratio_20','mean_magn','amplitude','trand_angle','skew','std']])
    Y = np.array(df1['target'])
    validation_size = 0.3
    seed = 2
    X_train, X_validation, Y_train, Y_validation = train_test_split(X, Y, test_size=validation_size, random_state=seed)
    model = XGBClassifier(random_state=34, seed=7, colsample_bytree=0.8, subsample=0.6, n_estimators=700)
    model.fit(X_train,Y_train)
    pred = model.predict(X_validation)
    Metrics=pd.DataFrame(columns=['Accuracy','precision','Roc','F1'],index=range(1))
    Metrics['Roc'][0]=roc_auc_score(pred.tolist(), Y_validation.tolist())
    Metrics['Accuracy'][0]=accuracy_score(Y_validation.tolist(), pred.tolist())
    Metrics['precision'][0]=precision_score(Y_validation.tolist(), pred.tolist())
    Metrics['F1'][0]=f1_score(Y_validation.tolist(), pred.tolist())
    return Metrics

In [120]:
def Metrics_classic(df):
    print('N(type1)=',len(df[df['target']==1]))
    print('N(type2)=',len(df[df['target']==0]))
    print('overall=',df.shape[0])
    print('best_random_res=',max(len(df[df['target']==1]),len(df[df['target']==0]))/df.shape[0] )
    X = np.array(df[['period-0','power-0','mean_magn','amplitude']])
    Y = np.array(df['target'])
    validation_size = 0.3
    seed = 2
    X_train, X_validation, Y_train, Y_validation = train_test_split(X, Y, test_size=validation_size, random_state=seed)
    model = XGBClassifier(random_state=34, seed=7, colsample_bytree=0.8, subsample=0.6, n_estimators=700)
    model.fit(X_train,Y_train)
    pred = model.predict(X_validation)
    Metrics=pd.DataFrame(columns=['Accuracy','precision','Roc','F1'],index=range(1))
    Metrics['Roc'][0]=roc_auc_score(pred.tolist(), Y_validation.tolist())
    Metrics['Accuracy'][0]=accuracy_score(Y_validation.tolist(), pred.tolist())
    Metrics['precision'][0]=precision_score(Y_validation.tolist(), pred.tolist())
    Metrics['F1'][0]=f1_score(Y_validation.tolist(), pred.tolist())
    return Metrics

In [132]:
#Cepheids 'zg' vs cepheids 'zr'
def compare(var_type,df1,df2):
    start = timeit.default_timer()
    n=len(df1[df1['Type']==var_type])+len(df2[df2['Type']==var_type])
    global df
    df=pd.DataFrame(columns=['ra','dec','Nobs','filter','Name'],index=range(n))
    df['period-0']=0.
    df['power-0']=0.
    df['period-1']=0.
    df['power-1']=0.
    df['period-2']=0.
    df['power-2']=0.
    df['period-3']=0.
    df['power-3']=0.
    df['period-4']=0.
    df['power-4']=0.
    df['abs_deviation_mag']=0.
    df['abs_deviation_pow']=0.
    df['freq_ratio_10']=0.
    df['freq_ratio_20']=0.    
    df['mean_magn']=0.
    df['amplitude']=0.
    df['trend_angle']=0.
    df['skew']=0.
    df['std']=0.
    df['target']=0
#     df=df.iloc[:n]

    names1=df1['Name'].tolist()
    names2=df2['Name'].tolist()
    k=0
    
    for i in range(df1.shape[0]):
        if df1['Type'][i]==var_type:
            df['target'][k]=0
            df['Name'][k]=df1['Name'][i]
            df['filter'][k]='zg'
            df['ra'][k]=df1['ra'][i]
            df['dec'][k]=df1['dec_x'][i]
            df['Nobs'][k]=df1['Nobs'][i]
            k=k+1
        
    for i in range(df2.shape[0]):  
        if df2['Type'][i]==var_type:
            df['target'][k]=1
            df['Name'][k]=df2['Name'][i]
            df['filter'][k]='zr'
            df['ra'][k]=df2['ra'][i]
            df['dec'][k]=df2['dec_x'][i]
            df['Nobs'][k]=df2['Nobs'][i]
            k=k+1

    names=df[df['Name'].notna()==True]['Name'].tolist()
    k=0
    for item in names:
        try:
            pathname=os.path.join(pathres,str(item)+".txt")
            with open(pathname) as f:
                res = f.read() #.replace("'", '"')
                res=json.loads(res)
                obj_mjd=[]
                obj_mag=[]
                obj_magerr=[]

                for obj in res.keys():
                    for ii in res[obj]['lc']:
                        obj_mjd.append(ii['mjd'])
                        obj_mag.append(ii['mag'])
                        obj_magerr.append(ii['magerr'])

                    frequency, power = LombScargle(obj_mjd, obj_mag, obj_magerr).autopower()
                    peaks=list(find_peaks(power))[0].tolist()
                    df['period-0'][k]=1/frequency[peaks[0]]
                    df['power-0'][k]=power[peaks[0]]
                    df['period-1'][k]=1/frequency[peaks[1]]
                    df['power-1'][k]=power[peaks[1]]
                    df['period-2'][k]=1/frequency[peaks[2]]
                    df['power-2'][k]=power[peaks[2]]
                    df['period-3'][k]=1/frequency[peaks[3]]
                    df['power-3'][k]=power[peaks[3]]
                    df['period-4'][k]=1/frequency[peaks[4]]
                    df['power-4'][k]=power[peaks[4]]
                    df['abs_deviation_mag'][k]=stats.median_absolute_deviation(obj_mag)
                    df['abs_deviation_pow'][k]=stats.median_absolute_deviation(power)
                    df['freq_ratio_10'][k]=frequency[peaks[1]]/frequency[peaks[0]]
                    df['freq_ratio_20'][k]=frequency[peaks[2]]/frequency[peaks[0]]
                    df['mean_magn'][k] = np.array(obj_mag).mean()
                    df['amplitude'][k] = (np.array(obj_mag).max() - np.array(obj_mag).min())/2
                    df['trend_angle'][k] = np.polyfit(np.array(obj_mjd), np.array(obj_mag), 1)[0] 
                    df['skew'][k] = scipy.stats.skew(np.array(obj_mag))
                    df['std'][k] = np.array(obj_mag).std()
                    k=k+1
        except:
            print(1)
            
    return  df 

In [100]:
df11=df1.iloc[:100]

In [84]:
peaks=list(find_peaks(power))[0].tolist()

In [87]:
frequency[peaks[0]]

0.003034613560929351

In [53]:
1/power[2]

1.0089609964498103

In [54]:
1/frequency[2]

329.53124999999994

In [190]:
df11['']

,index,ra,dec,Nobs,filter,Name,Type,period-0,power-0,period-1,...,abs_deviation_mag,abs_deviation_pow,freq_ratio_10,freq_ratio_20,mean_magn,amplitude,trand_angle,skew,std,target
0,0,12.847,34.37674,44.0,zg,10,M,329.531250,0.991119,56.815733,...,1.295051,0.032137,5.800000,8.600000,15.054762,2.8845,-0.021382,-1.735134,1.443471,0
1,1,344.524,48.75930,49.0,zg,104,SR,221.519886,0.362382,90.248843,...,0.200151,0.017484,2.454545,3.545455,12.882109,0.4230,-0.000421,-1.011818,0.220722,0
2,2,3.294,73.86323,52.0,zg,10406,M,94.695992,0.118074,66.000237,...,1.570073,0.018414,1.434783,1.869565,14.813367,2.7130,-0.005358,-0.813771,1.505385,0
3,3,331.369,62.50290,148.0,zg,10413,SRA,306.284722,0.857269,110.262500,...,0.274280,0.006967,2.777778,3.888889,12.881000,0.5405,-0.000688,-0.752244,0.274634,0
4,4,314.846,58.89249,54.0,zg,10422,M,251.512784,0.831938,70.939503,...,1.683492,0.023081,3.545455,4.636364,15.238900,2.0460,-0.001779,0.288561,1.316846,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,96,328.745,69.50928,54.0,zg,10857,LB,376.696429,0.770671,114.646739,...,0.111937,0.027159,3.285714,5.857143,13.438880,0.2665,0.001478,0.098958,0.137714,0
96,97,328.809,56.68865,85.0,zg,10858,LB,147.442555,0.160903,86.431843,...,0.031134,0.017335,1.705882,2.294118,12.742380,0.1330,-0.000122,2.124950,0.048908,0
97,98,328.864,61.28729,148.0,zg,10860,SRB,110.262500,0.210748,54.050245,...,0.080801,0.005851,2.040000,3.560000,13.633336,0.2255,0.000136,-0.315573,0.091708,0
98,99,328.894,67.13530,55.0,zg,10861,LB,292.990451,0.683362,125.567336,...,0.332102,0.015058,2.333333,3.444444,14.639077,0.5150,-0.001857,0.033496,0.274231,0


In [191]:
df1=Features('CEP',df1)
# df2=Features('CEP',df2)
# df3=Features('CEP',df3)

C:\Users\rtut6\Anaconda3\lib\site-packages\ipykernel_launcher.py:29: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
C:\Users\rtut6\Anaconda3\lib\site-packages\ipykernel_launcher.py:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
C:\Users\rtut6\Anaconda3\lib\site-packages\ipykernel_launcher.py:51: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
C:\Users\rtut6\Anaconda3\lib\site-packages\ipykernel_launcher.py:52: SettingWithCopyWarning: 
A value i

error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
erro

error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
error
erro

In [203]:
#delete all -inf and -nan
df1=df1[~df1.isin([np.nan, np.inf, -np.inf]).any(1)]

In [193]:
M1=Metrics(df1)
# M2=Metrics(df2)
# M3=Metrics(df2)
# r=[M1,M2,M3]
# m=pd.concat(r)

(76, 28)


ValueError: Only one class present in y_true. ROC AUC score is not defined in that case.

In [ ]:
M1

In [176]:
print(type(df1['period-0'][0]))
print(type(df1['power-0'][0]))

<class 'numpy.float64'>
<class 'numpy.float64'>


In [217]:
# grid search k value for SMOTE oversampling for imbalanced classification
# define dataset
# X, y = make_classification(n_samples=10000, n_features=17, n_redundant=0,
#                            n_clusters_per_class=4, weights=[0.99], flip_y=0, random_state=1)
X = np.array(df1[['period-0','power-0','period-1','power-1','period-2','power-2','period-3','power-3','period-4','power-4','abs_deviation_mag','abs_deviation_pow','freq_ratio_10','freq_ratio_20','mean_magn','amplitude','trand_angle','skew','std']])
y = np.array(df1['target'])
# values to evaluate
k_values = [1, 2, 3, 4, 5, 6, 7]
for k in k_values:
    # define pipeline
#     model = XGBClassifier(random_state=34, seed=7, colsample_bytree=0.8, subsample=0.6, n_estimators=400)
    model = DecisionTreeClassifier()
    over = SMOTE(sampling_strategy=0.3, k_neighbors=k)
    under = RandomUnderSampler(sampling_strategy=0.5)
    steps = [('over', over), ('under', under), ('model', model)]
    pipeline = Pipeline(steps=steps)
    # evaluate pipeline
    cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=3, random_state=1)
    scores = cross_val_score(pipeline, X, y, scoring='roc_auc', cv=cv, n_jobs=-1)
    score = mean(scores)
    print('> k=%d, Mean ROC AUC: %.3f' % (k, score))

> k=1, Mean ROC AUC: 0.503
> k=2, Mean ROC AUC: 0.509
> k=3, Mean ROC AUC: 0.494
> k=4, Mean ROC AUC: 0.482
> k=5, Mean ROC AUC: 0.493
> k=6, Mean ROC AUC: 0.494
> k=7, Mean ROC AUC: 0.494


In [146]:
X

array([[-2.84575800e-01, -7.44524435e-01, -1.39641918e+00, ...,
         1.42727432e+00, -2.13720849e+00,  2.81837056e-01],
       [-4.61340383e-01, -1.67780185e+00, -5.45282883e-01, ...,
         3.34525888e-01, -6.97098201e-01,  2.57724227e-01],
       [ 1.49589942e-01,  1.22243748e+00,  1.32033470e-01, ...,
        -2.66224637e-01, -7.46064552e-01,  2.02802542e+00],
       ...,
       [ 2.31655271e-01, -2.02956722e-01, -2.32136931e-02, ...,
         3.92334280e-02, -4.27006908e-01,  7.39316586e-01],
       [ 2.88015662e-01, -4.86650137e-01,  1.28878031e-03, ...,
         2.97134259e+00, -5.73172990e-01,  1.37007760e+00],
       [ 9.81062603e-01, -3.16362811e-01,  1.22739266e+00, ...,
         7.06925577e-01,  1.17618520e+00,  2.76471655e+00]])

In [149]:
y

array([0, 0, 0, ..., 0, 0, 0])

In [148]:
X = np.array(df1[['period-0','power-0','period-1','power-1','period-2','power-2','period-3','power-3','period-4','power-4','abs_deviation_mag','abs_deviation_pow','freq_ratio_10','freq_ratio_20','mean_magn','amplitude','trand_angle','skew','std']])
X

array([[ 3.29531250e+02,  9.91118590e-01,  5.68157328e+01, ...,
        -2.13823357e-02, -1.73513358e+00,  1.44347073e+00],
       [ 2.21519886e+02,  3.62382474e-01,  9.02488426e+01, ...,
        -4.20756612e-04, -1.01181816e+00,  2.20722060e-01],
       [ 9.46959918e+01,  1.18074112e-01,  6.60002367e+01, ...,
        -5.35757148e-03, -8.13771154e-01,  1.50538499e+00],
       ...,
       [ 1.69675481e+02,  3.70717612e-02,  8.82312500e+01, ...,
         1.12465660e-03,  1.65763303e-02,  1.10296183e+00],
       [ 1.39720553e+02,  8.23809799e-02,  9.55982730e+01, ...,
        -1.60632332e-03,  3.47043881e+00,  2.62603221e-01],
       [ 1.47052083e+02,  3.70893180e-02,  9.59035326e+01, ...,
        -1.93318952e-03,  1.17783325e+00,  3.94102274e-01]])

In [153]:
y = np.array(df1['target'])
y

array([1, 0, 1, ..., 0, 0, 0], dtype=int64)

In [137]:
# check version number
import imblearn
print(imblearn.__version__)

0.7.0


In [ ]:
# model = periodic.LombScargleAstroML(fit_period=False)
# model.optimizer.set(period_range=(1, 70), first_pass_coverage=90)
# model.fit(obj_mjd, obj_mag, obj_magerr)
# model.find_best_periods(n_periods=5,return_scores=True)
# model.periodogram(periods=True)
# periods = np.linspace(0.1, 10, 100)

# omega=2*np.pi/periods
# plt.scatter(scipy.signal.lombscargle(obj_mjd, obj_mag, omega), periods)

In [ ]:
# from astropy.timeseries import LombScargle
# frequency, power = LombScargle(obj_mjd, obj_mag, obj_magerr).autopower()
# frequency[np.argmax(power)]

# # plt.ylim(-1, 5)
# plt.plot(frequency, power)
# plt.ylabel('power')
# plt.xlabel('frequency cyclic')
# ax = plt.gca()
# ax.set_ylim(0,1)
# plt.title(obj)
# ax.scale('log')
# plt.ylim(-0.2, 0.6)
# plt.title('')
# print(1/frequency[np.argmax(power)])
# print(1/0.78)
# print(1/1.2)
# print(1/0.41)

In [ ]:
# from scipy.signal import find_peaks

# indices = find_peaks(power, threshold=0.001)

In [ ]:
# indices[0]

In [ ]:
# from gatspy.periodic import LombScargleFast
# model = LombScargleFast().fit(obj_mjd, obj_mag, obj_magerr)
# period, power = model.periodogram_auto(nyquist_factor=200)

# #                     D1=np.max(power[peak_data[0]])
# #                     ind1=power[peak_data[0]].argmax()
# #                     power[peak_data[0][ind1]]=0
# #                     D2=np.max(power[peak_data[0]])
# #                     ind2=power[peak_data[0]].argmax() 
# #                     object_return['max_fs'] =0

# Fs= 2*np.pi/(period[power.argmax()])
# Fs1= 2*np.pi/(period[peak_data[0][ind2]])

#### Cumulative curves for Nobs


In [ ]:
# # names=df1['Type'].value_counts()[:100].index.tolist()
# names=['DCEP','CEP','BCEP','UG','DCEPS','CEP(B)','CEPS','RR']
# # pdf = matplotlib.backends.backend_pdf.PdfPages("234.pdf")
# for item in names:
    
#     data1=df1[df1['Type']==item]['Nobs'].tolist()
#     data2=df2[df2['Type']==item]['Nobs'].tolist()
#     data3=df3[df3['Type']==item]['Nobs'].tolist()
    
#     values1, base1 = np.histogram(data1, bins=60)
#     values2, base2 = np.histogram(data2, bins=60)
#     values3, base3 = np.histogram(data3, bins=60)
    
#     cumulative1 = np.cumsum(values1)
#     cumulative2 = np.cumsum(values2)
#     cumulative3 = np.cumsum(values3)
    
#     #visualisation + saving
    
#     f=plt.figure(figsize=(8,5))
#     plt.gca().set_xlim([0,160])
#     ax = plt.gca()
# #     ax.set_ylim([0,2500])
#     ax.set_facecolor((1.0, 1, 1))
#     plt.plot(base1[:-1], len(data1)-cumulative1, c='green')
#     plt.plot(base2[:-1], len(data2)-cumulative2, c='red')
#     plt.plot(base3[:-1], len(data3)-cumulative3, c='blue',)
#     plt.legend(['ZG','ZR','SUMM'])
#     plt.title(item)
#     plt.grid(True,linestyle="--")

# #     pdf.savefig(f) #saving pdf
#     plt.show()
    
# # pdf.close()
